In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF

usermovie = pd.read_csv('../data/final_data.csv',sep=',')
movie_head = pd.read_csv('../data/movie_head.txt',sep=',')
#loan_inner=pd.merge(usermovie,movie_head,how='inner')

In [2]:
usermovie

,uid,mid,star,timeStamp
0,1722994,1306505,1,2007.0
1,1405477,10574468,1,2013.0
2,15849871,4910186,1,2011.0
3,1068524,1304643,1,2006.0
4,1307041,1851857,1,2008.0
5,1700174,3324130,1,2010.0
6,16124523,6860160,1,2013.0
7,1562429,1296339,1,2007.0
8,1475205,1907467,1,2008.0
9,1484972,5352872,1,2012.0


In [3]:
movie_head

,,mid,count
0,0,426,19133
1,1,172,17849
2,2,1040,15947
3,3,949,15567
4,4,936,15259
5,5,289,14866
6,6,1290,13970
7,7,630,13207
8,8,454,12694
9,9,723,12645


In [4]:
mid = usermovie['mid'].unique()
uid = usermovie['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
usermovie['mid'] = usermovie['mid'].map(lambda x:collect_mid_dencoder[x])
usermovie['uid'] = usermovie['uid'].map(lambda x:collect_uid_dencoder[x])

In [5]:
print(len(uid))
print(len(mid))

48608
72581
